In [ ]:
import sys
import os
import shutil
import numpy as np
import yaml
sys.path.append('/home/welling/git/tensorflow_apps/src')
from brainroller.traceneighbors import UsefulVtx
from brainroller.writegeom import writeVtkPolypoints
from cnn.constants import RAD_PIXELS
import spilltree3D
import pandas as pd
import pickle

In [ ]:
traceFile = '/tmp/useful_trace_neighborhoods.pkl'
skipFile = '/tmp/skips.txt'


In [ ]:
help(UsefulVtx)

In [ ]:
with open(traceFile, 'rb') as pklF:
    with open(skipFile, 'rb') as skipF:
        usefulVtxDict = UsefulVtx.load(pklF, 10000, skipF)


In [ ]:
spT = spilltree3D.SpTree(usefulVtxDict.values())


In [ ]:
assert spT.getNDim() == 3, 'spilltree3D was compiled incorrectly'

In [ ]:
fishCoreXSize = 1024
fishCoreYSize = 1024
fishCoreZSize = 4900
fishCoreXOffset = 4750. - (fishCoreXSize/2)
fishCoreYOffset = 2150. - (fishCoreYSize/2)
fishCoreZOffset = 4000

fishCoreBlockSize = 1024  # how many Z slices to actually load

scan_sz_V = np.asarray([41., 41., 41.])

In [ ]:
root_dir = '/home/welling/sshfshook'
#fish_cube_dir = 'pylon2_rescue/fish_cube_links'
fish_cube_dir = 'fish_cube_links'
output_root = '/home/welling/git/tensorflow_apps/src/joel'

In [ ]:
def write_all_verts_vtk(fname, sample_id, ctrCoords, usefulVtxDict, spill_tree, scan_sz_V, offsetV):
    if sample_id is None:
        ctrX, ctrY, ctrZ = ctrCoords
        ctrVtx = UsefulVtx(None, None, None, ctrX, ctrY, ctrZ)
    else:
        ctrVtx = usefulVtxDict[sample_id]
    max_sum_sqr = 0.25 * np.sum(scan_sz_V * scan_sz_V)
    sameObjLocL = []
    allLocL = []
    for uv, sepsqr in spill_tree.findApproxKNearest(1000, ctrVtx):
        allLocL.append(uv)
        if uv.objId == ctrVtx.objId:
            sameObjLocL.append(uv)
        if sepsqr > max_sum_sqr:
            break
    else:
        raise RuntimeError('More than 1000 verts in range!')
    print('%d nearby verts, %d in same object' % (len(allLocL), len(sameObjLocL)))
    realV = np.asarray(ctrVtx.realCoords)
    lclV = realV - offsetV

    x_off, y_off, z_off = offsetV + lclV
    shift1 = [-x_off, -y_off, -z_off]
    x2_off, y2_off, z2_off = - 0.5 * scan_sz_V
    shift2 = [-x2_off, -y2_off, -z2_off]
    writeVtkPolypoints(fname, allLocL, shift1=shift1, shift2=shift2, sig=[1.0, -1.0, 1.0])

def test_calcs(sample_id, empty=False):
    if empty:
        with open(os.path.join(root_dir, fish_cube_dir, 'empty_41_%d.yaml' % sample_id)) as f:
            infoD = yaml.safe_load(f)
    else:
        with open(os.path.join(root_dir, fish_cube_dir, 'block_41_%d.yaml' % sample_id)) as f:
            infoD = yaml.safe_load(f)
    ctrCoords = np.asarray(infoD['realCoords'])
    print('ctrCoords: ', ctrCoords)
    offsetV = np.asarray([int(np.floor(v)) for v in [fishCoreXOffset, fishCoreYOffset, fishCoreZOffset]])
    scanCoords = (ctrCoords - offsetV) - np.floor(0.5 * scan_sz_V)

    # Walk forward through the full dataset as necessary to get the target into the loaded part of the data
    nSkip = 0
    stepZ = fishCoreBlockSize // 2
    while (scanCoords[2] - nSkip) + scan_sz_V[2] > fishCoreBlockSize:
        nSkip += stepZ
    scanCoords[2] -= nSkip

    print('SCAN CONTROL PARAMETERS:')
    print('--data_block_dims=%d,%d,%d' % (fishCoreXSize, fishCoreYSize, fishCoreBlockSize))
    print('--data_block_offset=%d' % (nSkip * fishCoreXSize * fishCoreYSize))
    print('--scan_start=%s' % ','.join([str(int(v)) for v in np.floor(scanCoords)]))
    print('--scan_size=%s' % ','.join([str(int(v)) for v in scan_sz_V]))
    rslt_dir = os.path.join(output_root, 'test_%d' % sample_id)
    os.makedirs(rslt_dir, exist_ok=True)
    print('WRITING DATA TO: %s' % rslt_dir)
    for fname in infoD['bovFiles'] + [infoD['traceFile']]:
        shutil.copy(os.path.join(root_dir, fish_cube_dir, fname), rslt_dir)
    if empty:
        write_all_verts_vtk(os.path.join(rslt_dir, 'known_verts_%d.vtk' % sample_id),
                            None, ctrCoords, usefulVtxDict, spT, scan_sz_V, offsetV)
    else:
        write_all_verts_vtk(os.path.join(rslt_dir, 'known_verts_%d.vtk' % sample_id),
                            sample_id, ctrCoords, usefulVtxDict, spT, scan_sz_V, offsetV)
    print('\n')

In [ ]:
def write_blk_verts_vtk(fname, llfCoords, trbCoords, usefulVtxDict, offsetV):   
    lclV = 0.5*(trbCoords + llfCoords) - offsetV
    scan_sz_V = trbCoords - llfCoords
    print('llfCoords: ', llfCoords)
    print('trbCoords: ', trbCoords)
    print('offsetV: ', offsetV)
    print('lclV:', lclV)
    
    allLocL = []
    count = 0
    for ub in usefulVtxDict.values():
        if np.all(ub.realCoords >= llfCoords) and np.all(ub.realCoords <= trbCoords):
            #print(ub.realCoords, ub.objId)
            allLocL.append(ub)
            count += 1
    print('including {} points'.format(count))

    x_off, y_off, z_off = offsetV + lclV
    shift1 = [-x_off, -y_off, -z_off]
    x2_off, y2_off, z2_off = - 0.5 * scan_sz_V
    shift2 = [-x2_off, -y2_off, -z2_off]
    print('shift1: ', shift1)
    print('shift2: ', shift2)
    writeVtkPolypoints(fname, allLocL, shift1=shift1, shift2=shift2, sig=[1.0, -1.0, 1.0])

def test_blk_calcs(blockOffset, scanStartTpl, scanSzTpl):
    scan_sz_V = np.asarray(scanSzTpl)
    scan_start_V = np.asarray(scanStartTpl)
    scan_ctr_V = scan_start_V + 0.5 * scan_sz_V
    #ctrCoords = np.asarray(infoD['realCoords'])
    offset_V = np.asarray([int(np.floor(v)) for v in [fishCoreXOffset, fishCoreYOffset, fishCoreZOffset]])
    print('offset_V: ', offset_V)
    offset_V += np.asarray([0, 0, blockOffset//(fishCoreXSize * fishCoreYSize)])
    print('corrected offset_V: ', offset_V)
    real_start_V = scan_start_V + offset_V
    print('real_start_V: ', real_start_V)
    write_blk_verts_vtk(os.path.join(output_root, 'test_verts.vtk'),
                        real_start_V, real_start_V + scan_sz_V, usefulVtxDict,
                        offset_V)

test_blk_calcs(2621440000, (384,384,32), (448,320, 128))

In [ ]:
test_calcs(4901045)
#test_calcs(994, empty=True)
#test_calcs(983792)
#test_calcs(2453393)
  

In [ ]:
test_blk_calcs()

```
'absCoords': [4635.3, 1936.44, 5321.0], 
'realCoords': [4635.3, 1936.44, 5285.0],
--data_block_dims=1024,1024,1024
--data_block_offset=536870912
--scan_start=377,278,753
--scan_size=41,41,41
WRITING DATA TO: /home/welling/git/tensorflow_apps/src/joel/test_1010391
192 nearby verts, 20 in same object
```

In [ ]:
help(UsefulVtx)

In [ ]:
write_all_verts_vtk('/tmp/junk/try.vtk', None, [4871.0, 2174.5,4158.0], usefulVtxDict, spT, scan_sz_V, [4238, 1638, 6500])

```python

observedCoords=[249,152.5, 126]
vtxId=4901045
realCoords=[4870.76, 2189.5, 6658.0]


offset_V:  [4238 1638 4000]
corrected offset_V:  [4238 1638 6500]
real_start_V:  [4622 2022 6532]
llfCoords:  [4622 2022 6532]
trbCoords:  [5070 2342 6660]
offsetV:  [4238 1638 6500]
lclV: [608. 544.  96.]
including 8672 points
shift1:  [-4846.0, -2182.0, -6596.0]
shift2:  [224.0, 160.0, 64.0]
    
    blockGen = BlockGenerator(rMax, edgeLen, maxL, 
                              fakeVtxDict, fishCoreFile,
                              fishCoreXSize, fishCoreYSize, fishCoreZSize,
                              baseName=baseName)

                blockGen.writeBlock(sampleId, 
                                    {'xOffset': fishCoreXOffset,
                                     'yOffset': fishCoreYOffset,
                                     'zOffset': fishCoreZOffset})

        self.sampler = ArraySampler(edgeLen, fishCoreFile,
                                    fishCoreXSize, fishCoreYSize,
                                    fishCoreZSize)

        grabBlock(sampleVtx, self.usefulVtxDict, self.sampler,
                  self.transformer, self.rMax, self.edgeLen,
                  self.maxL, sampLocInfo['xOffset'],
                  sampLocInfo['yOffset'], sampLocInfo['zOffset'],
                  baseName=self.baseName)

    xLoc = sampleVtx.x
    yLoc = sampleVtx.y
    zLoc = sampleVtx.z
    dCube = sampler.sample({'xLoc': xLoc, 'yLoc': yLoc, 'zLoc': zLoc,
                            'xOffset': xOffset, 
                            'yOffset': yOffset,
                            'zOffset': zOffset})
    writeBOV(bovBaseName, dCube, 1.0, 1.0, 1.0, sig=(1.0, -1.0, 1.0))

    if sig[0] < 0:
        data = data[::-1, :, :]
    if sig[1] < 0:
        data = data[:, ::-1, :]
    if sig[2] < 0:
        data = data[:, :, ::-1]
    np.ravel(data, order='F').tofile(fnameRoot + ext)


```

In [ ]:
def backCalc(observedCoords):
    oX, oY, oZ = observedCoords
    scanStart, scanSz = (384,384,32), (448,320, 128)
    offset_V = [4238, 1638, 6500] # corrected
    real_start_V =  [4622, 2022, 6532]
    llfCoords =  [4622, 2022, 6532]
    trbCoords = [5070, 2342, 6660]
    offsetV = [4238, 1638, 6500]
    
    gblX = oX + llfCoords[0]
    gblY = (scanSz[1] - oY) + llfCoords[1]
    gblZ = oZ + llfCoords[2]
    return (gblX, gblY, gblZ)

    #lclV: [608. 544.  96.]
    #including 8672 points
    #shift1:  [-4846.0, -2182.0, -6596.0]
    #shift2:  [224.0, 160.0, 64.0]
    
observedCoords=[249,152.5, 126]
vtxId=4901045
realCoords=[4870.76, 2189.5, 6658.0]
print(backCalc(observedCoords))


In [ ]:
errantDF = pd.read_csv('errant_empty_points.csv')

In [ ]:
errantGlobalCoordL = []
for idx, rec in errantDF.iterrows():
    errantGlobalCoordL.append(backCalc((rec.x, rec.y, rec.z)))
with open('errant_empty_locs.pkl', 'wb') as f:
    pickle.dump(errantGlobalCoordL, f)